In [1]:
import pathlib
import sys

In [2]:
import pandas as pd

In [3]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
interim_dir_path = data_dir_path / "interim"
interim_path = interim_dir_path / "interim.parquet"
train_path = interim_dir_path / "train.parquet"
src_dir_path = root_dir_path / "src"

In [4]:
sys.path.append(str(src_dir_path))

In [5]:
from package.feature_extraction import *
from package.utils import *

In [6]:
interim = pd.read_parquet(interim_path)

In [7]:
interim.dropna(inplace=True, subset=["demand"])

In [8]:
interim.reset_index(drop=True, inplace=True)

In [9]:
extract_features(interim)

In [10]:
reduce_memory_usage(interim)

In [11]:
interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 29 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   id                               object        
 1   item_id                          int16         
 2   store_id                         int8          
 3   dept_id                          int8          
 4   cat_id                           int8          
 5   state_id                         int8          
 6   d                                object        
 7   demand                           int16         
 8   date                             datetime64[ns]
 9   event_name_1                     int8          
 10  event_type_1                     int8          
 11  event_name_2                     int8          
 12  event_type_2                     int8          
 13  snap_CA                          int8          
 14  snap_TX                         

In [12]:
interim.to_parquet(train_path)